In [1]:
import torch
from tqdm import tqdm
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoConfig
import os
from datasets import load_from_disk
os.environ['TRANSFORMERS_CACHE'] = '../transformers_cache/'


In [2]:
data_test = load_from_disk("../datasets/gsm8k/test/")
data_test

Dataset({
    features: ['question', 'answer'],
    num_rows: 1319
})

In [3]:
data_train = load_from_disk("../datasets/gsm8k/train/")
data_train

Dataset({
    features: ['question', 'answer'],
    num_rows: 896
})

In [4]:
hf_token = os.getenv("hf_token")
model_name = "meta-llama/Llama-3.2-1B-Instruct"
config = AutoConfig.from_pretrained(model_name, token=hf_token)
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, config=config,cache_dir='../transformers_cache')
model = AutoModelForCausalLM.from_pretrained(model_name, token=hf_token, config=config,cache_dir='../transformers_cache')
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'left'
# model.resize_token_embeddings(len(tokenizer))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [8]:
'<|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nYour response should end with \"The final answer is [answer]\" where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThere are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The final answer is 6<|eot_id|>

In [7]:
# userContent += '\nQ: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?'
# userContent += '\nA: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. #### 6\n'
# userContent += '\nQ: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?'
# userContent += '\nA: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. #### 5\n'
# userContent += '\nQ: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?'
# userContent += '\nA: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. #### 39\n'
# userContent += '\nQ: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?'
# userContent += '\nA: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. #### 8\n'
# userContent += '\nQ: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?'
# userContent += '\nA: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. #### 9\n'
# userContent += '\nQ: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?'
# userContent += '\nA: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. #### 29\n'
# userContent += '\nQ: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?'
# userContent += '\nA: Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. #### 33\n'
# userContent += '\nQ: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?'
# userContent += '\nA: Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. #### 8\n'


In [13]:
# questions = [
#     'There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?',
#     'If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?',
#     'Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?',
#     'Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?',
#     'Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?',
#     'There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?',
#     'Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?',
#     'Olivia has $23. She bought five bagels for $3 each. How much money does she have left?'
    
# ]

In [14]:
# answers = [
#     'There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. #### 6',
#     'There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. #### 5',
#     'Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. #### 39',
#     'Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. #### 8',
#     'Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. #### 9',
#     'There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. #### 29',
#     ' Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. #### 33',
#     'Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. #### 8'

# ]

In [15]:
# user = '\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: $ans \nYour response should end with \"#### [answer]\" where [answer] is the response to the problem.'

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 09 Jan 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Given the following problem, reason and give a final answer to the problem.
Problem: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
Your response should end with "#### [answer]" where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. #### 6<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [17]:
prompt = "<|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nYour response should end with \'#### [answer]\' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThere are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. #### 6<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?\nYour response should end with \'#### [answer]\' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThere are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. #### 5<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?\nYour response should end with \'#### [answer]\' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nOriginally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. #### 39<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?\nYour response should end with \'#### [answer]\' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nJason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. #### 8<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?\nYour response should end with \'#### [answer]\' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nShawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. #### 9<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?\nYour response should end with \'#### [answer]\' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThere were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. #### 29<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?\nYour response should end with \'#### [answer]\' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nMichael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. #### 33<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?\nYour response should end with \'#### [answer]\' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nOlivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. #### 8<|eot_id|>"

In [ ]:
<|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: Lori wants to buy a $320.00 pair of shoes and a matching belt that is $32.00. Her part-time job pays her $8.00 an hour. How many hours will she have to work before she can make her purchase?\nYour response should end with \"The final answer is [answer]\" where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n" ]

In [21]:
from tqdm import tqdm
model.eval()
generated_outputs=[]
# Adjust batch size according to your GPU memory capacity
# With vram=80G, 1B - 64, 3B - 64, 8B - 32
batch_size=1
for i in tqdm(range(0, len(data_test["question"]), batch_size), desc="Processing questions"):
    batch_questions = data_test["question"][i:i+batch_size]
    inputs = []
    for q in batch_questions:
        inputs.append(prompt + f'<|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: {q}\nYour response should end with \'#### [answer]\' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n') 
    print("INPUT:\n", inputs)
    tokenized_inputs = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True)
    tokenized_inputs.to(device)

    with torch.no_grad():
        output = model.generate(**tokenized_inputs, max_length=2000, num_return_sequences=1, pad_token_id=tokenizer.pad_token_id, do_sample=True, temperature=0.1, top_p=0.95)
        # output = model.generate(**tokenized_inputs, max_new_tokens=256, num_return_sequences=1, pad_token_id=tokenizer.pad_token_id, do_sample=True, temperature=0.1, top_p=0.95)
    
    for j, o in enumerate(output):
        generated_text = tokenizer.decode(o, skip_special_tokens=True)
        print("OUTPUT:\n",generated_text)
        answer = generated_text.split("Your response should end with '#### [answer]' where [answer] is the response to the problem.assistant")[-1]
        # generated_outputs.append({"input": inputs[j], "output": generated_text, "question": batch_questions[j], "answer":answer})
        # print(batch_questions[j])
        print("Answer:\n",answer)
    break

Processing questions:   0%|          | 0/1319 [00:00<?, ?it/s]

INPUT:
 ["<|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nYour response should end with '#### [answer]' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThere are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. #### 6<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?\nYour response should end with '#### [answer]' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThere a

Processing questions:   0%|          | 0/1319 [00:02<?, ?it/s]

OUTPUT:
 user

Given the following problem, reason and give a final answer to the problem.
Problem: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
Your response should end with '#### [answer]' where [answer] is the response to the problem.assistant

There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. #### 6user

Given the following problem, reason and give a final answer to the problem.
Problem: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
Your response should end with '#### [answer]' where [answer] is the response to the problem.assistant

There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. #### 5user

Given the following problem, reason and give a final answer to the problem.
Problem: Leah had 32 chocolates and her sister

In [8]:
userContent='\n\nBelow are few example question and answer pairs\n\n'

userContent += "Here your job is to answer a math question. "
userContent += f"Your question will appear after 8 demonstrations of similar math tasks. "
userContent += "As in those demonstrations, you must generate both a step-by-step reasoning and a final answer. "
userContent += "Perform a simple action, e.g., a single mathematical operation, at each step of your reasoning. "
userContent += "Your final answer must contain only a number and no additional text. "
userContent += "State your final answer after ####. "

userContent

'\n\nBelow are few example question and answer pairs\n\nHere your job is to answer a math question. Your question will appear after 8 demonstrations of similar math tasks. As in those demonstrations, you must generate both a step-by-step reasoning and a final answer. Perform a simple action, e.g., a single mathematical operation, at each step of your reasoning. Your final answer must contain only a number and no additional text. State your final answer after ####. '

In [12]:
chat = [
    {"role":"system", "content": "You are a helpful assistant."},
    {"role":"user", "content":userContent}
]
chat = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True, include_metadata=False)
print(chat)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 10 Dec 2024

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below are few example question and answer pairs

Here your job is to answer a math question. Your question will appear after 8 demonstrations of similar math tasks. As in those demonstrations, you must generate both a step-by-step reasoning and a final answer. Perform a simple action, e.g., a single mathematical operation, at each step of your reasoning. Your final answer must contain only a number and no additional text. State your final answer after ####.<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [4]:
# 8-shot prompt
fewShotPrompt=f'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n'
fewShotPrompt+='\n\nBelow are few example question and answer pairs\n\n'

fewShotPrompt += "Here your job is to answer a math question. "
fewShotPrompt += f"Your question will appear after 8 demonstrations of similar math tasks. "
fewShotPrompt += "As in those demonstrations, you must generate both a step-by-step reasoning and a final answer. "
fewShotPrompt += "Perform a simple action, e.g., a single mathematical operation, at each step of your reasoning. "
fewShotPrompt += "Your final answer must contain only a number and no additional text. "
fewShotPrompt += "State your final answer after ####. "

print(fewShotPrompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Here your job is to answer a math question. Your question will appear after 8 demonstrations of similar math tasks. As in those demonstrations, you must generate both a step-by-step reasoning and a final answer. Perform a simple action, e.g., a single mathematical operation, at each step of your reasoning. Your final answer must contain only a number and no additional text. State your final answer after ####. 


In [5]:
for i in range(9):
    fewShotPrompt+=f'Q: {data_train["question"][i]}\nA: {data_train["answer"][i]}\n\n'

In [11]:
from tqdm import tqdm
model.eval()
generated_outputs=[]
# Adjust batch size according to your GPU memory capacity
# With vram=80G, 1B - 64, 3B - 64, 8B - 32
batch_size=2
for i in tqdm(range(0, len(data_test["question"]), batch_size), desc="Processing questions"):
    batch_questions = data_test["question"][i:i+batch_size]
    inputs = [fewShotPrompt+"Now, solve the below question following the instructions given above. \n\nQ: "+q+"\nA: <|eot_id|><|start_header_id|>assistant<|end_header_id|>" for q in batch_questions]
    # inputs = [fewShotPrompt+"Now, Follow the same format for reasoning and stating your final answer as above examples and Answer the below question\n\nQ: "+q+"<|eot_id|><|start_header_id|>assistant<|end_header_id|>" for q in batch_questions]
    tokenized_inputs = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True)
    tokenized_inputs.to(device)

    with torch.no_grad():
        output = model.generate(**tokenized_inputs, max_length=2000, num_return_sequences=1, pad_token_id=tokenizer.pad_token_id, do_sample=True, temperature=0.1, top_p=0.95)
        # output = model.generate(**tokenized_inputs, max_new_tokens=256, num_return_sequences=1, pad_token_id=tokenizer.pad_token_id, do_sample=True, temperature=0.1, top_p=0.95)
   
    for j, o in enumerate(output):
        generated_text = tokenizer.decode(o, skip_special_tokens=True)
        answer = generated_text.split("A: assistant")[-1]
        generated_outputs.append({"input": inputs[j], "output": generated_text, "question": batch_questions[j], "answer":answer})
        print(batch_questions[j])
        print(answer)
    break

Processing questions:   0%|          | 0/660 [00:04<?, ?it/s]

Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?


First, find the number of eggs laid per day that Janet sells at the market: 16 eggs - 3 eggs - 4 eggs = <<16-3-4=9>>9 eggs
Then multiply the number of eggs sold by the price per egg: 9 eggs * $2/egg = $<<9*2=18>>18
#### 18
A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?


To find the total number of bolts needed, I need to first find the number of bolts of white fiber. Since it takes half as much white fiber as blue fiber, I will divide the number of blue fiber bolts by 2.

2 bolts of blue fiber / 2 = 1 bolt of white fiber

Now, I will add the number of bolts of blue fiber and white fiber to find the total number of bolts.

2 bolts of blue fiber + 1 b

In [7]:

from tqdm import tqdm
batch_size=2
for i in tqdm(range(0, len(data_test["question"]), batch_size), desc="Processing questions"):
    batch_questions = data_test["question"][i:i+batch_size]
    inputs = [fewShotPrompt+"Now, solve the below question following the instructions given above. \n\nQ: "+q+"\nA: <|eot_id|><|start_header_id|>assistant<|end_header_id|>" for q in batch_questions]
    break

Processing questions:   0%|          | 0/660 [00:00<?, ?it/s]


In [9]:
print(inputs[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Here your job is to answer a math question. Your question will appear after 8 demonstrations of similar math tasks. As in those demonstrations, you must generate both a step-by-step reasoning and a final answer. Perform a simple action, e.g., a single mathematical operation, at each step of your reasoning. Your final answer must contain only a number and no additional text. State your final answer after ####. Q: Bobby wanted pancakes for breakfast.  The recipe on the box makes 21 pancakes.  While he ate 5 pancakes, his dog jumped up and was able to eat 7 before being caught.  How many pancakes does Bobby have left?
A: Bobby ate 5 pancakes and his dog ate 7 so 5+7 = <<5+7=12>>12
The recipe makes 21 pancakes and 12 were eaten so 21-12 = <<21-12=9>>9 pancakes were left
#### 9

Q: A store gives a 10% discount for the amount of the sell that was over $

In [10]:
print(inputs[1])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Here your job is to answer a math question. Your question will appear after 8 demonstrations of similar math tasks. As in those demonstrations, you must generate both a step-by-step reasoning and a final answer. Perform a simple action, e.g., a single mathematical operation, at each step of your reasoning. Your final answer must contain only a number and no additional text. State your final answer after ####. Q: Bobby wanted pancakes for breakfast.  The recipe on the box makes 21 pancakes.  While he ate 5 pancakes, his dog jumped up and was able to eat 7 before being caught.  How many pancakes does Bobby have left?
A: Bobby ate 5 pancakes and his dog ate 7 so 5+7 = <<5+7=12>>12
The recipe makes 21 pancakes and 12 were eaten so 21-12 = <<21-12=9>>9 pancakes were left
#### 9

Q: A store gives a 10% discount for the amount of the sell that was over $